In [1]:
import random
import pandas as pd
from surprise.model_selection import KFold
from surprise import Dataset, SVD, SVDpp
from surprise.model_selection import GridSearchCV
from surprise import KNNBasic
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
import numpy

In [2]:
userIDs = {}
gameIDs = {}
interactions = []

f = open("train.json")
f.readline()
for l in f:
    d = eval(l)
    u = d['userID']
    g = d['gameID']
    h = d['hours_transformed']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in gameIDs: gameIDs[g] = len(gameIDs)
    interactions.append((u,g,h))

In [3]:
random.shuffle(interactions)

In [4]:
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [5]:
df = pd.DataFrame(interactionsTrain,columns = ["userID","gameID","hours"])

In [6]:
dis1 = df.groupby('gameID')['userID'].count().sort_values()#.clip(upper=50)
dis1

gameID
g10701910      7
g17749950      7
g67585399      8
g41077930      8
g29170259      9
            ... 
g11862712    671
g40499587    713
g75228197    803
g05463839    847
g10773791    993
Name: userID, Length: 2437, dtype: int64

In [7]:
dis2 = df.groupby('userID')['gameID'].count().sort_values()#.clip(upper=50)
dis2

userID
u68490160      1
u49097895      1
u99532246      1
u91746794      1
u22588287      1
            ... 
u61683676    178
u04495704    189
u04893836    194
u22405118    206
u88835772    212
Name: gameID, Length: 6708, dtype: int64

In [8]:
min_book_ratings = 10
filter_books = df['gameID'].value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

min_user_ratings = 10
filter_users = df['userID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

In [9]:
df_new = df[(df['gameID'].isin(filter_books)) & (df['userID'].isin(filter_users))]

In [10]:

reader = Reader()
data = Dataset.load_from_df(df_new, reader=reader)

In [15]:
param_grid = {"n_epochs": [10], "lr_all": [0.1,0.01,0.001], "reg_all": [0.1,0.01,0.001,0.0001,0.00001]}#,0.4, 0.6, 0.8]}
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse"], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

1.847527899718526
{'n_epochs': 10, 'lr_all': 0.01, 'reg_all': 0.1}


In [16]:
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

In [17]:
preds = []
y = []
for u,g,h in interactionsTest:
    preds.append(algo.predict(u, g).est)
    y.append(h)
numpy.mean(numpy.square(numpy.array(y) - numpy.array(preds)))

3.342435326720206